## imports

In [1]:
import os,time,sys
import shutil


import ghtest.param_suggestor as ps
from ghtest import make_test, scan, suggest, write_module
from ghtest.tests_writer import TestArtifact
from ghtest import cassette_sanitizer as cs

## format

In [2]:
# set print behavior
from IPython.display import display, Javascript
display(Javascript("""
document.querySelectorAll('.jp-OutputArea-child').forEach(e => e.style.maxHeight = 'none');
"""))

<IPython.core.display.Javascript object>

## constants

In [3]:
# input
cassette_dir='tests/cassettes'
test_dir='testdata_tests/'
#src_folder='../temp_folder/src/apigh/'
src_folder='testdata'
#test_folder='../temp_folder/tests/'
safe_switch='GHTEST_ASSUME_SAFE'
is_unsafe='1'
remove_current_tests=True
remove_all_tests=True
remove_cassettes=False
remove_history=True

## switches

In [4]:
os.environ[safe_switch]=is_unsafe

## remove stale test data

In [5]:
# remove param_db and param history
if remove_history:
    remove_param_db=True
    param_db_path='../../.local/state/ghtest/param_db.json'
    param_hist_path='../../.local/state/ghtest/param_history.json'
    pps=[param_db_path, param_hist_path]
    for pt in pps:
        if os.path.isfile(pt):
            os.remove(pt)
            bn=os.path.basename(pt)
            print(f'{bn} removed')
print(f'no history db removed')

no history db removed


## scan

In [6]:
# scan 
scs=scan(src_folder)
#sct=scan(test_folder)len(scs)#, len(sct)

### filter non-api and test methods

In [7]:
funcs=scs
#funcs=[f for f in funcs if not f.qualname.startswith('_')]
funcs=[f for f in funcs if not f.qualname.startswith('test_')]
#funcs=scs
#funcs=funcs[0:2]
for func in funcs:
    print(func.qualname )

_remove_auth
_get_env_value
_get_user
_get_token
_get_headers
_paginate
get_commits
list_repos
create_repo
delete_repo
get_repo_data


## suggest 

In [8]:
sps=[]
for func in funcs:
    sp=suggest(func,test_functions=None, literal_only=True)
    #print(sp.module, sp.qualname)
    for ps in sp.param_sets:
        pass
        #print(ps)
    sps.append(sp)

### change parameters

In [9]:
# for security, we change 'coron' to coron7
param_update={'name':'coron7'}
for sp in sps:
    updated_param_set=[]
    for op in sp.param_sets:
        for k,v in param_update.items():
            if k in op:
                op=op|param_update
        updated_param_set.append(op)
    sp.param_sets=updated_param_set
    #print(sp.qualname)
    #print(sp.param_sets)
    #print()

## make tests

### remove old cassettes and tests

In [13]:
gts=[]
cfs=[os.path.join(cassette_dir,f) for f in os.listdir(cassette_dir)]
nr=0
for sp in sps:
    gt=make_test(suggestion=sp, cassette_dir=cassette_dir)
    gts.append(gt)
    cp=gt.cassette_path
    bn=os.path.splitext(cp)[0]
    for f in cfs:
        if remove_cassettes and bn in f and os.path.isfile(f):
            os.remove(f)
            nr+=1
print(f'{nr} files deleted from {cassette_dir}')

0 files deleted from tests/cassettes


In [14]:
# remove previous tests
funcnames=[f.qualname  for f in funcs]
nf,nd=0,0
for file in os.listdir(test_dir):
    path=os.path.join(test_dir, file)
    if os.path.isfile(path):
        if remove_all_tests or any(f in file for f in funcnames) and remove_current_tests:
            os.remove(path)
            nf+=1
    elif os.path.isdir(path):
        shutil.rmtree(path)
        nd+=1
print(f'{nf} files, {nd} folders deleted from {test_dir}')

0 files, 0 folders deleted from testdata_tests/


### create cassettes

In [15]:
trs=[]
for gt in gts:
    try:
        tr=gt.test_callable()
        trs.append(tr)
    except Exception as e:
        print(str(e))

### sanitize cassettes

In [16]:
args=['tests/cassettes',]
r=cs.main(args)

updated 53 cassette(s) out of 53


### record results

In [17]:
trs=[]
for gt in gts:
    try:
        tr=gt.test_callable()
        trs.append(tr)
    except Exception as e:
        print(str(e))

In [18]:
### write tests

In [19]:
artifacts=[]
rrs=[]
for sp,tr in zip(sps, trs):
    artifacts.append(TestArtifact(suggestion=sp, run=tr))
r=write_module(artifacts=artifacts, output_dir=test_dir)
r

TestWriterResult(test_modules=[PosixPath('testdata_tests/test_generated_0.py'), PosixPath('testdata_tests/test_generated_1.py'), PosixPath('testdata_tests/test_generated_2.py'), PosixPath('testdata_tests/test_generated_3.py'), PosixPath('testdata_tests/test_generated_4.py'), PosixPath('testdata_tests/test_generated_5.py'), PosixPath('testdata_tests/test_generated_6.py'), PosixPath('testdata_tests/test_generated_7.py')], scenario_modules=[PosixPath('testdata_tests/scenarios/scenario_repo_0.py')])